In [ ]:
import numpy as np
import pandas as pd
#import ml libs
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
np.random.seed(0)
#natural language learning tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction import text
import string 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#import ignore warning
import warnings
warnings.filterwarnings('ignore')
#import cross validation and random forest and adaboost
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
# import confusion matrix and accuracy score 
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score



In [ ]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [ ]:
path = "/drive/My Drive/ML in Agriculture/allrecipes.csv"
df= pd.read_csv(path)
df

,Unnamed: 0,name,ingredients,recipe_cuisine,rating,rating_count,keywords,CuisineLabel
0,0,Paneer Bhurjee with English Peas,4 tablespoons canola oil 1/2 tablespoon musta...,Indian,0.0,0.0,"Cilantro, Cumin, Garlic, Ginger, Lemon Juice, ...",2.0
1,1,Lemon Bars With a Salty Olive Oil Crust,2 1/2 cups (320 grams) all-purpose flour 1/2 ...,American,0.0,0.0,"Lemon, Bake, Cookie, 5 Ingredients or Fewer ,D...",0.0
2,2,Pickle-Brined Chicken Nuggets,1/2 cup pickle juice 1 tablespoon light brown...,American,4.0,1.0,"Back to School, Kid-Friendly, Chicken ,Dinner",0.0
3,3,Grandma Best Raspberry Jam,14 pounds raspberries 8 boxes of pectin (Gra...,American,4.0,2.0,"Raspberry, Pickle Preserve, On the Cheap, Mak...",0.0
4,4,"The Chunkiest, Herbiest, Greenest Guacamole","1 clove garlic, grated 1/2 lime, juiced 1 ...",Mexican,4.0,3.0,"Guacamole, Avocado, Party, Quick and Easy ,App...",5.0
...,...,...,...,...,...,...,...,...
22640,17386,Vietnamese Dipping Sauce,2 tablespoons fresh lime juice 3 tablespoons ...,Vietnamese,3.0,2.0,Sauce Citrus Garlic No-Cook Quick & Easy ...,2.0
22641,17387,Beef Stew with Curry and Lemongrass,4 cups water 2 extra-large beef bouillon cube...,Vietnamese,3.0,36.0,Soup/Stew Beef Mushroom Tomato Vegetable ...,2.0
22642,17388,Vietnamese Beef Soup,8 ounces bean thread noodles* 10 cups canned ...,Vietnamese,3.0,22.0,Soup/Stew Beef Leafy Green Kid-Friendly Qu...,2.0
22643,17389,Vietnamese Fried Rice,2 bunches scallions 3 large carrots 2 cups f...,Vietnamese,0.0,33.0,Onion Rice Vegetable Side Stir-Fry Vegeta...,2.0


In [ ]:
#set up data for model 
df=df.drop('Unnamed: 0', axis=1)
df.rating=df.rating.fillna(0)
df['name']=df['name'].fillna('unknown')
df['rating_count']=df['rating_count'].fillna(0)
target_rating=(round(df['rating'])).astype(str)
target_cuisine=df['CuisineLabel']
target_ratingcount=df['rating_count']
#combine recipe_cuisine with keywords
df=df.drop('recipe_cuisine', axis=1)

In [ ]:
uncleaned_df=df[['name', 'ingredients', 'keywords']]
uncleaned_df

,name,ingredients,keywords
0,Paneer Bhurjee with English Peas,4 tablespoons canola oil 1/2 tablespoon musta...,"Cilantro, Cumin, Garlic, Ginger, Lemon Juice, ..."
1,Lemon Bars With a Salty Olive Oil Crust,2 1/2 cups (320 grams) all-purpose flour 1/2 ...,"Lemon, Bake, Cookie, 5 Ingredients or Fewer ,D..."
2,Pickle-Brined Chicken Nuggets,1/2 cup pickle juice 1 tablespoon light brown...,"Back to School, Kid-Friendly, Chicken ,Dinner"
3,Grandma Best Raspberry Jam,14 pounds raspberries 8 boxes of pectin (Gra...,"Raspberry, Pickle Preserve, On the Cheap, Mak..."
4,"The Chunkiest, Herbiest, Greenest Guacamole","1 clove garlic, grated 1/2 lime, juiced 1 ...","Guacamole, Avocado, Party, Quick and Easy ,App..."
...,...,...,...
22640,Vietnamese Dipping Sauce,2 tablespoons fresh lime juice 3 tablespoons ...,Sauce Citrus Garlic No-Cook Quick & Easy ...
22641,Beef Stew with Curry and Lemongrass,4 cups water 2 extra-large beef bouillon cube...,Soup/Stew Beef Mushroom Tomato Vegetable ...
22642,Vietnamese Beef Soup,8 ounces bean thread noodles* 10 cups canned ...,Soup/Stew Beef Leafy Green Kid-Friendly Qu...
22643,Vietnamese Fried Rice,2 bunches scallions 3 large carrots 2 cups f...,Onion Rice Vegetable Side Stir-Fry Vegeta...


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#create stopword extractor for cleaning, and function for cleaning words
punc = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%",'/', 1,2,3,4,5,6,7,8,9,0, '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '1/2', '1/3', '2/3', '1/4', '3/4' ]
stop_words = text.ENGLISH_STOP_WORDS.union(punc)
def recipe_cleaner(recipe):
    recipe = word_tokenize(recipe)
    nopunc=[word.lower() for word in recipe if word not in stop_words]
    nopunc=' '.join(nopunc)
    return [word for word in nopunc.split()]
#clean words for the text columns (ingredients, name, keywords)
df['ingredients']=df['ingredients'].apply(lambda x:recipe_cleaner(x))
df['name']=df['name'].apply(lambda x:recipe_cleaner(x))
df['keywords']=df['keywords'].apply(lambda x:recipe_cleaner(x))

TypeError: ignored

In [ ]:
cleaned_df=df[['name', 'ingredients', 'keywords']]
cleaned_df

,name,ingredients,keywords
0,Paneer Bhurjee with English Peas,"[tablespoons, canola, oil, tablespoon, mustard...","Cilantro, Cumin, Garlic, Ginger, Lemon Juice, ..."
1,Lemon Bars With a Salty Olive Oil Crust,"[cups, 320, grams, all-purpose, flour, cup, 10...","Lemon, Bake, Cookie, 5 Ingredients or Fewer ,D..."
2,Pickle-Brined Chicken Nuggets,"[cup, pickle, juice, tablespoon, light, brown,...","Back to School, Kid-Friendly, Chicken ,Dinner"
3,Grandma Best Raspberry Jam,"[14, pounds, raspberries, boxes, pectin, grand...","Raspberry, Pickle Preserve, On the Cheap, Mak..."
4,"The Chunkiest, Herbiest, Greenest Guacamole","[clove, garlic, grated, lime, juiced, ripe, av...","Guacamole, Avocado, Party, Quick and Easy ,App..."
...,...,...,...
22640,Vietnamese Dipping Sauce,"[tablespoons, fresh, lime, juice, tablespoons,...",Sauce Citrus Garlic No-Cook Quick & Easy ...
22641,Beef Stew with Curry and Lemongrass,"[cups, water, extra-large, beef, bouillon, cub...",Soup/Stew Beef Mushroom Tomato Vegetable ...
22642,Vietnamese Beef Soup,"[ounces, bean, thread, noodles*, 10, cups, can...",Soup/Stew Beef Leafy Green Kid-Friendly Qu...
22643,Vietnamese Fried Rice,"[bunches, scallions, large, carrots, cups, fre...",Onion Rice Vegetable Side Stir-Fry Vegeta...


In [ ]:
#create count vectorizer 
bow = CountVectorizer(stop_words='english')

In [ ]:
#name&ingredients&keywords vs cuisinetype 
name_and_ingredients_and_keywords=pd.DataFrame()
name_and_ingredients_and_keywords=df['name'].astype(str).str.cat(df[['ingredients', 'keywords', 'rating', 'rating_count']].astype(str), sep=' ,')
y=target_cuisine
#count vectorize text domaine 
X=bow.fit_transform(name_and_ingredients_and_keywords)
#split into test/train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
#create bagged tree model and fit the data 
bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_depth=100), n_estimators=150)
bagged_tree.fit(X_train, y_train)
#examine training and testing score 


BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=100,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,
 

In [ ]:
training_score=bagged_tree.score(X_train,y_train)
training_score

0.9662729079266946

In [ ]:
testing_score=bagged_tree.score(X_test,y_test)
testing_score

0.895120335614926

In [ ]:
#predict cuisine type
def predict_cuisine_type(text): 
    text=[str(text)]
    vec_string=bow.transform(text)
    predict=bagged_tree.predict(vec_string)
    predict_to_string=str(predict.astype('int')).replace('[', '').replace(']', '')
    if predict_to_string == '0': 
        prediction='American'
    elif predict_to_string == '1': 
        prediction='European'
    elif predict_to_string == '2': 
        prediction='Asian'
    elif predict_to_string == '3': 
        prediction='MiddleEastern/African'
    elif predict_to_string == '5': 
        prediction='SouthAmerican '
    else : 
        prediction= 'unknown'    
        
    return prediction

In [ ]:
vegan_palak_paneer=['14-ounce', 'block', 'extra-firm', 'tofu', 'tablespoons', 'lemon', 'juice', 'tablespoons', 'water', 'teaspoon', 'salt', 'tablespoons', 'nutritional', 'yeast', '10', 'ounces', 'cups', 'spinach', 'leaves', 'tablespoon', 'olive', 'oil', 'poblano', 'serrano', 'chili', 'seeds', 'removed', 'diced', 'white', 'yellow', 'onion', 'chopped', 'tablespoon', 'grated', 'minced', 'fresh', 'ginger', 'cloves', 'garlic', 'minced', 'tomato', 'chopped', 'cups', 'use', 'diced', 'tomatoes', 'draining', 'excess', 'liquid', 'adding', 'recipe', 'teaspoon', 'salt', 'teaspoon', 'garam', 'masala', 'teaspoon', 'ground', 'cumin', 'teaspoons', 'ground', 'coriander', 'dash', 'red', 'pepper', 'flakes', 'tablespoon', 'lime', 'juice', 'cup', 'cashew', 'cream', 'headnote', 'coconut', 'milk', 'cup', 'basmati', 'rice', 'serving', 'optional', 'handful', 'chopped', 'cilantro', 'chopped', 'green', 'onions', 'serving', 'optional']
print('Input:', "Vegan Palak Paneer",vegan_palak_paneer)
print('')
print('Correct type: Asian')
print('')
print('Prediction:', predict_cuisine_type(vegan_palak_paneer))


Input: Vegan Palak Paneer ['14-ounce', 'block', 'extra-firm', 'tofu', 'tablespoons', 'lemon', 'juice', 'tablespoons', 'water', 'teaspoon', 'salt', 'tablespoons', 'nutritional', 'yeast', '10', 'ounces', 'cups', 'spinach', 'leaves', 'tablespoon', 'olive', 'oil', 'poblano', 'serrano', 'chili', 'seeds', 'removed', 'diced', 'white', 'yellow', 'onion', 'chopped', 'tablespoon', 'grated', 'minced', 'fresh', 'ginger', 'cloves', 'garlic', 'minced', 'tomato', 'chopped', 'cups', 'use', 'diced', 'tomatoes', 'draining', 'excess', 'liquid', 'adding', 'recipe', 'teaspoon', 'salt', 'teaspoon', 'garam', 'masala', 'teaspoon', 'ground', 'cumin', 'teaspoons', 'ground', 'coriander', 'dash', 'red', 'pepper', 'flakes', 'tablespoon', 'lime', 'juice', 'cup', 'cashew', 'cream', 'headnote', 'coconut', 'milk', 'cup', 'basmati', 'rice', 'serving', 'optional', 'handful', 'chopped', 'cilantro', 'chopped', 'green', 'onions', 'serving', 'optional']

Correct type: Asian

Prediction: Asian


In [ ]:
#name,keywords,cuisinetype, ingredients vs rating 
y=target_rating.astype(str)
name_keywords_ingredients=df['name'].astype(str).str.cat(df[['ingredients', 'keywords', 'CuisineLabel']].astype(str), sep=' ,')
#count vectorize scaler  
X=bow.fit_transform(name_keywords_ingredients)
#train_test_split target and scaler 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
forest = RandomForestClassifier(n_estimators=100, max_depth= 100)
# random  over sampler to account for the under represented sample 
ros=RandomOverSampler(random_state=42)
X_train, y_train =ros.fit_sample(X_train, y_train)
forest.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=100, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
training_score=forest.score(X_train,y_train)
training_score

0.98831491987658

In [ ]:
testing_score=forest.score(X_test,y_test)
testing_score

0.7151689114594834